# Healthcare Database Python Code

In [29]:
# Import all necessary libraries and packages
import os
import pandas as pd
import json
import pprint
import requests
import requests.exceptions
import pymysql
from sqlalchemy import create_engine

## Load CSV File and Convert to DF

In [30]:
data_dir = os.path.join(os.getcwd(), 'downloads')
data_file = os.path.join(data_dir, 'admission_data.csv')

try:
    admissions_df = pd.read_csv(data_file, header=0, index_col=0)
    result_head = admissions_df.head()
except FileNotFoundError as errf:
     print(f"Error: File not found - {errf}")
     result_head = pd.DataFrame()
except pd.errors.EmptyDataError as erre:
    print(f"Error: The CSV file is empty - {erre}")
    result_head = pd.DataFrame()
except pd.errors.ParserError as errp:
    print(f"Error: There was an error parsing the CSV file - {errp}")
    result_head = pd.DataFrame()
except Exception as err:
    print(f"An unexpected error occurred - {err}")
    result_head = pd.DataFrame()
    

result_head

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
case_id,,,,,,,,,,,,,,,,,
1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


### Export as SQL File

In [31]:
# Get information for connection
try:
    with open('env_info.json') as f:
        info = json.load(f)
except FileNotFoundError:
    print("Error: 'env_info.json' file not found.")
except json.JSONDecodeError:
    print("Error: unable to parse 'env_info.json'. Make sure this is a valid JSON file.")
try:
    db_user = info['DB_USER']
    db_password = info['DB_PASSWORD']
    db_host = info['DB_HOST']
    db_port = info['DB_PORT']
    db_name = info['DB_NAME']
    
# Check to make sure environment variables are there, return error statement if not
except KeyError as e:
    print(f"Error: Missing environment variable {e}")

# Create a MySQL engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Export the Dataframe to MySQL
try:
    admissions_df.to_sql(name = 'Admissions', con = engine, index = True, if_exists = 'replace')
    print("DataFrame sucessfully exported to MySQL!")
except Exception as e:
    print(f"Error: Unable to export DataFrame to MySQL. {e}")

DataFrame sucessfully exported to MySQL!


## Get Data from Hospital API

In [32]:
def get_hospital_info(state):
    url = f"https://www.communitybenefitinsight.org/api/get_hospitals.php?state={state}"
    
    try:
        response = requests.request("GET", url)
        response.raise_for_status()
        data = response.json()
        
        if "error" in data:
            return "Error in API response: {data['error']}"
        
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except json.JSONDecodeError as errj:
        return "JSON Decode Error: " + repr(errj)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)


    return data

### Get the Hospital Data Output (JSON)

In [33]:
state = input('Enter a state using its two-letter abbreviation: ')
hospital_data = get_hospital_info(state)

Enter a state using its two-letter abbreviation: NJ


### Convert to Dataframe

In [34]:
df = pd.DataFrame(hospital_data)
df

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1903,1491,010649794,Hackensack University Medical Center,Hackensack University Medical Center,30 Prospect Avenue,Hackensack,NJ,07601,34003,781,N,Y,N,Y,310001,Bergen County,>299 beds,"November 20, 2023"
1,1904,1493,851296795,Newark Beth Israel Medical Center,Newark Beth Israel Medical Center,201 Lyons Avenue,Newark,NJ,07112,34013,665,N,Y,N,Y,310002,Essex County,>299 beds,"November 20, 2023"
2,1905,1491,010649794,Palisades Medical Center,Palisades Medical Center,7600 River Road,North Bergen,NJ,07047,34017,202,N,Y,N,N,310003,Hudson County,100-299 beds,"November 20, 2023"
3,1906,1495,221537688,Hunterdon Medical Center,Hunterdon Medical Center,2100 Wescott Drive,Flemington,NJ,08822,34019,184,N,Y,N,Y,310005,Hunterdon County,100-299 beds,"November 20, 2023"
4,1907,1496,221494446,St Marys General Hospital,St Marys Hospital - Passaic,350 Boulevard,Passaic,NJ,07055,34031,286,N,Y,N,N,310006,Passaic County,100-299 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1962,1508,210634532,Virtua - Marlton,,Rt 73 Brick Rd,Marlton,NJ,08053,34005,204,N,Y,N,N,310117,Burlington County,100-299 beds,"November 20, 2023"
60,1963,1542,223284894,Meadowlands Hospital Medical Center,,55 Meadowlands Pkwy,Secaucus,NJ,07094,34017,208,N,Y,N,N,310118,Hudson County,100-299 beds,"November 20, 2023"
61,1964,1522,223319886,St Clares Hospital - Sussex,St Clares Hospital - Sussex,20 Walnut Street,Sussex,NJ,07461,34037,106,Y,Y,N,N,310120,Sussex County,100-299 beds,"November 20, 2023"
62,1965,1491,010649794,Hackensack Umc At Pascack Valley,Hackensack Umc At Pascack Valley,250 Old Hook Road,Westwood,NJ,07675,34003,128,N,Y,N,Y,310130,Bergen County,100-299 beds,"November 20, 2023"


### Export as SQL File

In [35]:
# Get information for connection
try:
    with open('env_info.json') as f:
        info = json.load(f)
except FileNotFoundError:
    print("Error: 'env_info.json' file not found.")
except json.JSONDecodeError:
    print("Error: unable to parse 'env_info.json'. Make sure this is a valid JSON file.")
try:
    db_user = info['DB_USER']
    db_password = info['DB_PASSWORD']
    db_host = info['DB_HOST']
    db_port = info['DB_PORT']
    db_name = info['DB_NAME']
    
# Check to make sure environment variables are there, return error statement if not
except KeyError as e:
    print(f"Error: Missing environment variable {e}")

# Create a MySQL engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Export the Dataframe to MySQL
try:
    df.to_sql(name = 'Hospitals', con = engine, index = False, if_exists = 'replace')
    print("DataFrame sucessfully exported to MySQL!")
except Exception as e:
    print(f"Error: Unable to export DataFrame to MySQL. {e}")

DataFrame sucessfully exported to MySQL!
